In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../data/nhanes/medications-use.csv');

In [3]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [4]:
SELECT Med.nomeMedicamento, COUNT(Uso.CodMedicamento) FROM Medicamento Med, UsoMedicamento Uso 
WHERE Uso.CodMedicamento = Med.Codigo GROUP BY Uso.CodMedicamento;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [5]:
DROP VIEW MaisUsado IF EXISTS;
CREATE VIEW MaisUsado AS SELECT Uso.CodMedicamento, COUNT(Uso.CodMedicamento) QTD FROM UsoMedicamento Uso 
GROUP BY Uso.CodMedicamento;
SELECT CodMedicamento FROM MaisUsado WHERE QTD = (SELECT MAX(QTD) FROM MaisUsado);

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [6]:
SELECT CodMedicamento FROM 
(SELECT Uso.CodMedicamento, COUNT(Uso.CodMedicamento) AS QTD FROM UsoMedicamento Uso 
GROUP BY Uso.CodMedicamento) 
WHERE QTD = (SELECT MAX(QTD) FROM 
(SELECT Uso.CodMedicamento, COUNT(Uso.CodMedicamento) AS QTD FROM UsoMedicamento Uso 
GROUP BY Uso.CodMedicamento));

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [7]:
DROP VIEW MaisUsado IF EXISTS;
CREATE VIEW MaisUsado AS SELECT Uso.CodMedicamento, COUNT(Uso.CodMedicamento) QTD FROM UsoMedicamento Uso GROUP BY Uso.CodMedicamento;
SELECT Med.nomeMedicamento FROM Medicamento Med
WHERE Med.Codigo = (SELECT CodMedicamento FROM MaisUsado WHERE QTD = (SELECT MAX(QTD) FROM MaisUsado));

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [8]:
SELECT AVG(Id) FROM (SELECT COUNT(Uso.IdPessoa) AS Id FROM UsoMedicamento Uso 
                     GROUP BY Uso.IdPessoa);

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [9]:
SELECT Id FROM 
(SELECT COUNT(Uso.IdPessoa) AS Qtd, Uso.IdPessoa AS Id FROM UsoMedicamento Uso GROUP BY Uso.IdPessoa)
WHERE Qtd > (SELECT AVG(Id) FROM 
(SELECT COUNT(Uso.IdPessoa) AS Id FROM UsoMedicamento Uso GROUP BY Uso.IdPessoa));

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [10]:
SELECT Pessoa.Idade AS Idade, COUNT(Uso.IdPessoa) AS QtdMedicamento
FROM UsoMedicamento Uso, Pessoa WHERE Uso.IdPessoa = Pessoa.Id GROUP BY Uso.IdPessoa ;

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [20]:
DROP VIEW MedAB IF EXISTS;
CREATE VIEW MedAB AS SELECT DISTINCT Uso.IdPessoa, GREATEST(Uso.CodMedicamento, Uso2.CodMedicamento), LEAST(Uso.CodMedicamento, Uso2.CodMedicamento)
    FROM  UsoMedicamento Uso, UsoMedicamento Uso2
    WHERE Uso2.IdPessoa = Uso.IdPessoa AND Uso.CodMedicamento <> Uso2.CodMedicamento;
--SELECT * FROM MedAB;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [12]:
SELECT COUNT(IdPessoa) FROM MedAB GROUP BY Cod1, Cod2;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [13]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [14]:
UPDATE Matriz SET IdadeAte60 = true WHERE Idade < 60;
--SELECT * FROM Matriz;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [15]:
UPDATE Matriz SET MaisUmAnoMedicamento = true WHERE Matriz.Id = 
(SELECT DISTINCT IdPessoa FROM UsoMedicamento Uso WHERE Uso.DiasUso >365 AND IdPessoa = Matriz.Id);
--SELECT * FROM Matriz;

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [16]:
DROP VIEW IdNumMedAcimaMedia IF EXISTS;
CREATE VIEW IdNumMedAcimaMedia AS (
    SELECT Id FROM 
(SELECT COUNT(Uso.IdPessoa) AS Qtd, Uso.IdPessoa AS Id FROM UsoMedicamento Uso GROUP BY Uso.IdPessoa)
WHERE Qtd > (SELECT AVG(Id) FROM 
(SELECT COUNT(Uso.IdPessoa) AS Id FROM UsoMedicamento Uso GROUP BY Uso.IdPessoa))
);
             
UPDATE Matriz SET MedicamentosAcimaMedia = true WHERE Matriz.Id = 
(SELECT DISTINCT Id FROM IdNumMedAcimaMedia Uso WHERE Id = Matriz.Id);
SELECT * FROM Matriz;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

Obter a porcentagem de pessoas acima de 60 anos que tomam medicamento a mais de um ano e compará-la com a porcentangem de pessoa abaixo de 60 anos que tomam medicamento a mais de um ano.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [26]:
SELECT(
(CAST((SELECT COUNT(ID) FROM Matriz WHERE IdadeAte60 = false AND MaisUmAnoMedicamento = true) AS float))/
(CAST((SELECT COUNT(ID) FROM Matriz WHERE IdadeAte60 = false) AS float))
);

0.7821835958718087

In [27]:
SELECT(
(CAST((SELECT COUNT(ID) FROM Matriz WHERE IdadeAte60 = true AND MaisUmAnoMedicamento = true) AS float))/
(CAST((SELECT COUNT(ID) FROM Matriz WHERE IdadeAte60 = true) AS float))
);

0.18010559155267578